### Requrired Libraries 

In [1]:
from flask import Flask, request, render_template

### Importing Libraries

In [2]:
import os
import cv2
import numpy as np
from deepface import DeepFace
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.applications.resnet50 import preprocess_input

### Load Preprocess Imagees

In [3]:
app = Flask(__name__)

UPLOAD_FOLDER = "static/uploads"
DATABASE_FOLDER = r"archive (3)\lfw-deepfunneled\lfw-deepfunneled"  # Update to the LFW dataset folder

In [4]:
def verify_face(image1, image2):
    """Compares two images using DeepFace and returns similarity."""
    try:
        result = DeepFace.verify(image1, image2, model_name="Facenet")
        return result['verified'], result['distance']
    except Exception as e:
        print("Error during face verification:", e)
        return False, None

In [5]:
def identify_missing_person(missing_person_img, database_path):
    """Compares a missing person's image against a database of found persons."""
    for person_folder in os.listdir(database_path):
        person_folder_path = os.path.join(database_path, person_folder)
        print(person_folder_path)
        if os.path.isdir(person_folder_path):  # Check if it's a directory
            for person_img in os.listdir(person_folder_path):
                person_img_path = os.path.join(person_folder_path, person_img)
                match, distance = verify_face(missing_person_img, person_img_path)

                if match:
                    print(f"Match Found: {person_img_path} (Distance: {distance:.4f})")
                    return person_img_path  # Return the matched image path
    print("No match found.")
    return None



In [6]:
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/upload", methods=["POST"])
def upload():
    if "file" not in request.files:
        return "No file uploaded", 400

    file = request.files["file"]
    file_path = os.path.join(UPLOAD_FOLDER, file.filename)
    file.save(file_path)

    matched_image = identify_missing_person(file_path, DATABASE_FOLDER)

    if matched_image:
        return render_template("result.html", match=matched_image)
    else:
        return render_template("result.html", match=None)


In [ ]:
if __name__ == "__main__":
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [18/Feb/2025 21:35:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2025 21:35:48] "GET /favicon.ico HTTP/1.1" 404 -


archive (3)\lfw-deepfunneled\lfw-deepfunneled\Aaron_Eckhart


127.0.0.1 - - [18/Feb/2025 21:36:09] "POST /upload HTTP/1.1" 200 -


Match Found: archive (3)\lfw-deepfunneled\lfw-deepfunneled\Aaron_Eckhart\Aaron_Eckhart_0001.jpg (Distance: 0.0000)


### Verifying

### Comparing missing person photos with database